# 전체 과정
- 데이터 준비 > 전처리 >  피처선택 > 모델 학습 > 모델의 성능, 결과 평가 > 예측값 확인

## 데이터 준비

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("241212_01_MLlib_clustering").getOrCreate()

24/12/13 13:42:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
# 데이터 생성
data = [
    (0, 0, 4.0),  # user 0 rated item 0 with 4.0
    (0, 1, 2.0),
    (1, 1, 3.0),
    (1, 2, 1.0),
    (2, 0, 5.0),
    (2, 2, 4.0)
]

columns = ["user_id", "item_id", "rating"]

## 피처선택

In [3]:
rating_df = spark.createDataFrame(data, columns)
rating_df.show()

[Stage 0:>                                                          (0 + 1) / 1]

+-------+-------+------+
|user_id|item_id|rating|
+-------+-------+------+
|      0|      0|   4.0|
|      0|      1|   2.0|
|      1|      1|   3.0|
|      1|      2|   1.0|
|      2|      0|   5.0|
|      2|      2|   4.0|
+-------+-------+------+



In [5]:
# user, item - rating 정보를 >> 사용자 그룹을 만든다. > 모델 > 예측 결과 - 그룹

## 전처리

In [6]:
user_item_matrix = rating_df.groupBy("user_id").pivot("item_id").avg("rating").fillna(0)

In [7]:
user_item_matrix.show()

+-------+---+---+---+
|user_id|  0|  1|  2|
+-------+---+---+---+
|      0|4.0|2.0|0.0|
|      1|0.0|3.0|1.0|
|      2|5.0|0.0|4.0|
+-------+---+---+---+



## 피처 벡터

In [8]:
from pyspark.ml.feature import VectorAssembler

In [9]:
assembler = VectorAssembler(inputCols=["0","1","2"], outputCol="features")

In [11]:
user_features = assembler.transform(user_item_matrix)
user_features.show()

+-------+---+---+---+-------------+
|user_id|  0|  1|  2|     features|
+-------+---+---+---+-------------+
|      0|4.0|2.0|0.0|[4.0,2.0,0.0]|
|      1|0.0|3.0|1.0|[0.0,3.0,1.0]|
|      2|5.0|0.0|4.0|[5.0,0.0,4.0]|
+-------+---+---+---+-------------+



## 모델 생성 > 학습

In [12]:
from pyspark.ml.clustering import KMeans

#모델 생성
kmeans = KMeans(k=2, seed=1, featuresCol="features", predictionCol="cluster")
kmeans

KMeans_6fa051fe0ab8

In [13]:
#모델 학습
model = kmeans.fit(user_features)
model

24/12/13 14:13:36 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
24/12/13 14:13:36 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
                                                                                

KMeansModel: uid=KMeans_6fa051fe0ab8, k=2, distanceMeasure=euclidean, numFeatures=3

In [14]:
#모델을 이용한 예측
clusters = model.transform(user_features)

In [15]:
# 유저의 클러스터링 결과
clusters.show()

+-------+---+---+---+-------------+-------+
|user_id|  0|  1|  2|     features|cluster|
+-------+---+---+---+-------------+-------+
|      0|4.0|2.0|0.0|[4.0,2.0,0.0]|      0|
|      1|0.0|3.0|1.0|[0.0,3.0,1.0]|      0|
|      2|5.0|0.0|4.0|[5.0,0.0,4.0]|      1|
+-------+---+---+---+-------------+-------+



## 클러스터링 과정 요약
> 사용자 그룹화 : 유사한 취향의 사용자끼리 그룹으로 묶어주는 것.  
> 아이템 그룹화 : 아이템 간의 군집화를 통해 사용자에게 추천해 줄 수 있음.

➕ 요약

1. **데이터 프레임 생성**
주어진 데이터와 열 이름을 사용하여 데이터프레임을 생성하고 출력함.

2. **유저-아이템 매트릭스 생성**
평점 데이터를 피벗 테이블 형태로 변환하여 사용자별 아이템 평균 평점을 매트릭스로 만듦.

3. **피처 벡터 생성**
각 사용자의 평점 데이터를 피처 벡터로 변환함.

4. **KMeans 클러스터링 모델 생성**
KMeans 클러스터링 모델을 생성하고 학습시킴.

5. **모델을 이용한 예측**
학습된 모델을 사용해 사용자 데이터를 클러스터링함.

6. **추가 학습 정보**
- 평균 점수를 매트릭스로 만든다:
사용자가 동일한 아이템에 여러 번 평가했을 때, 그 평가 점수의 평균 값을 사용하는 것임.

- 클러스터의 기준:
KMeans 클러스터링에서 각 데이터 포인트는 임의로 선택된 초기 중심에 할당되고, 그 후 데이터와 중심 사이의 거리를 계산하여 중심을 반복적으로 조정함. 이렇게 중심을 조정하는 과정에서 데이터가 속한 클러스터도 변할 수 있음.

In [16]:
spark.stop()